In [1]:
from airflow import DAG
from airflow.operators.empty import EmptyOperator
from airflow.providers.apache.spark.operators.spark_submit import SparkSubmitOperator
from operators.custom_operators import CustomSparkSubmitOperator
import pendulum
import yaml
from pathlib import Path

default_args = {
    'owner': 'vinh'
    , 'email': 'ltvinh1101@gmail.com'
    , 'email_on_failure': True
    , 'email_on_retry': True
    , 'retries': 2
    , 'retry_delay': pendulum.duration(seconds = 1)
}
   
with DAG(
    dag_id = 'logistics_dag_v2'
    , description = 'This is a dag'
    , start_date = pendulum.now(tz = 'Asia/Ho_Chi_Minh')
    , default_args = default_args
    , catchup = False
    , schedule = '@daily'
    , tags = ['logistics', 'mysql']
) as dag:
    #START
    start = EmptyOperator(task_id = 'start')

    # config_path = Path.cwd() / 'config/config.yaml'
    # with open(config_path) as file:
    #     config = yaml.safe_load(file)
    #     packages = config['spark']['packages']
        
    process_enirched_apps = [
        'process_enriched_users'
        , 'process_enriched_drivers'
        , 'process_enriched_orders'
        , 'process_enriched_paymentss'
        , 'process_enriched_shipments'
    ]
    
    processe_enriched_operators = []
    
    for app_name in process_enirched_apps:
        temp = SparkSubmitOperator(
            task_id = app_name
            , conn_id = 'spark-conn'
            , application = app_name
  
        )
        processe_enriched_operators.append(temp)
        
    #END
    end = EmptyOperator(task_id = 'end')

    start >> processe_enriched_operators >> end
    

SyntaxError: invalid syntax (2682591987.py, line 50)

In [ ]:
from airflow.providers.apache.spark.operators.spark_submit import SparkSubmitOperator
from airflow.models import DAG
from datetime import datetime
import yaml

class MyCustomSparkOperator(SparkSubmitOperator):
    def __init__(self, config_path, *args, **kwargs):
        # Only store the path during initialization; don't read the file here.
        self.config_path = config_path
        super().__init__(*args, **kwargs)

    def execute(self, context):
        # Now load the configuration at execution time.
        with open(self.config_path, "r") as f:
            dynamic_config = yaml.safe_load(f)
        
        # Update parameters dynamically, if needed.
        if "conf" in dynamic_config:
            self.conf = dynamic_config["conf"]
        if "application" in dynamic_config:
            self.application = dynamic_config["application"]
        # Proceed with the normal execution flow.
        return super().execute(context)

# Define your DAG.
dag = DAG(
    "my_custom_spark_dag",
    start_date=datetime(2024, 2, 6),
    schedule_interval="@once",
    catchup=False,
)

# Use the custom operator.
spark_task = MyCustomSparkOperator(
    task_id="submit_spark_job",
    config_path="/path/to/config.yaml",
    dag=dag,
)


In [2]:
from operators.custom_operators import CustomSparkSubmitOperator

In [1]:
import requests

In [2]:
url = 'http://livy:8998/sessions'
response = requests.get(
    url = url
    , params = {
        'from': 0
        , 'size': 1
    }
)


In [8]:
import requests
import json

In [9]:
url = 'http://livy:8998/sessions'
headers = {'Content-Type': 'application/json'}
data = {
    'kind':'pyspark'
    , 'name': 'test1'
    , 'driverMemory': '512m'
    , 'driverCores': 1
    , 'executorMemory': '512m'
    , 'executorCores': 1
    , 'numExecutors': 1
    , 'ttl': '10m'
    , 'heartbeatTimeoutInSecond': 10
}
response = requests.post(url, data=json.dumps(data), headers=headers)
response

<Response [201]>

In [77]:
url = 'http://livy:8998/sessions'
headers = {'Content-Type': 'application/json'}
data = {
    'kind': 'pyspark'
    , 'name': 'logistics'
    , 'driverMemory': '512m'
    , 'driverCores': 1
    , 'executorMemory': '512m'
    , 'executorCores': 1
    , 'numExecutors': 1
}

response = requests.post(url, json.dumps(data), headers=headers)

In [78]:
response

<Response [201]>

In [18]:
json.dumps(params)

'{"kind": "pyspark", "name": "test", "driverMemory": "512m", "driverCores": 1, "executorMemory": "512m", "executorCores": 1, "numExecutors": 1}'

In [30]:
import textwrap

In [ ]:
POST /sessions/{sessionId}/statements

In [14]:
import textwrap

In [83]:
url = 'http://livy:8998/sessions/0/statements'
data = {
    'code': textwrap.dedent('''
        for i in range(10):
            print(i)
    ''')
    , 'kind': 'pyspark'
}

response = requests.post(url, data=json.dumps(data))

In [84]:
response.json()

{'id': 0,
 'code': '\nfor i in range(10):\n    print(i)\n',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [24]:
url = 'http://livy:8998/sessions/1'
response = requests.delete(url)

In [ ]:
url = 'http://http:8998/sessions/2'

requests.get(url)

In [ ]:
2025-02-13 09:34:15 25/02/13 02:34:15 WARN LivySparkUtils$: Spark version (3,5) is greater then the maximum version (3,0) supported by Livy, will choose Scala version 2.12 instead, please specify manually if it is the expected Scala version you want

In [87]:
url = 'http://livy:8998/batches'
headers = {'Content-Type': 'application/json'}
data = {
    'file': '/src/logistics_project_v2/spark_dataframe_app.py'
}
response = requests.post(url, json.dumps(data), headers=headers)

In [88]:
response.json()

{'id': 0,
 'name': None,
 'owner': None,
 'proxyUser': None,
 'state': 'running',
 'appId': None,
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [72]:
url = 'http://livy:8998/batches/1'

requests.delete(url)

<Response [200]>

In [35]:
url = 'http://livy:8998/batches'

requests.get(url).json()

{'from': 0,
 'total': 1,
 'sessions': [{'id': 0,
   'name': None,
   'owner': None,
   'proxyUser': None,
   'state': 'starting',
   'appId': None,
   'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
   'log': []}]}

In [49]:
url = 'http://livy:8998/sessions/0'
requests.delete(url)

<Response [200]>

In [54]:
url = 'http://livy:8998/batches/0'

requests.delete(url)

<Response [200]>